Loads relevant packages, instantiates spotify credentials manager

In [3]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import time
import random
import requests
import regex
from string import punctuation
client_id = "6f8bb3e64c2944d586037b9907043601"
client_secret = "81f5f44660364831be8d9118ccb51785"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Setting up the genres and the famous artists from each genre to grab off Spotify

In [4]:
singers = pd.read_csv('singers.csv')
singers['Genre'].value_counts()

Pop          173
Hip-Hop      151
Rock         146
Blues        124
Rap          124
Classical     92
EDM           49
Qawwali       10
Ghazal        10
Metal         10
Name: Genre, dtype: int64

Getting samples of Top 10 songs from each singer

In [42]:
for i in range(len(singers)):
    artist = singers.loc[i,'Singers']
    result = sp.search(artist)
    singers.loc[i, 'URI'] = result['tracks']['items'][0]['artists'][0]['uri']

Getting artist id, track id and preview link for later downloading

In [51]:
artist_uri = []
track_uri = []
track_preview = []
for artist in singers['URI']:
    results = sp.artist_top_tracks(artist)
    for track in results['tracks'][:10]:
        try:  
            artist_uri.append(artist)
            track_uri.append(track['uri'])
            track_preview.append(track['preview_url'])
        except:
            next    

Joining the newly scraped data into a dataframe

In [52]:
track_df = pd.DataFrame()
track_df['Artist'] = artist_uri
track_df['TrackURI'] = track_uri
track_df['Preview'] = track_preview

Merging the genre/artist data with newly scraped data from Spotify

In [54]:
total_df = pd.merge(singers, track_df, how = 'outer', left_on='URI', right_on = 'Artist')

Removing NAs

In [71]:
total_df2 = total_df.dropna()

Extracting Audio Features from Each Song that Spotify Calculates (Warnimg takes 60min+ to run, better off just full_spotify.csv)

In [77]:
df = pd.DataFrame(total_df2['TrackURI'])
df.columns = ['TrackURI']

for i in range(0,df.shape[0]):
  try:
    time.sleep(random.uniform(3, 6))
    URI = df.TrackURI[i]
    features = sp.audio_features(URI)
    track = sp.track(URI)
    df.loc[i, 'track'] = track['name']
    df.loc[i,'acousticness'] = features[0]['acousticness']
    df.loc[i,'instrumentalness'] = features[0]['instrumentalness']
    df.loc[i,'energy'] = features[0]['energy']
    df.loc[i,'speechiness'] = features[0]['speechiness']
    df.loc[i,'liveness'] = features[0]['liveness']
    df.loc[i,'loudness'] = features[0]['loudness']
    df.loc[i,'danceability'] = features[0]['danceability']
    df.loc[i,'tempo'] = features[0]['tempo']
    df.loc[i,'valence'] = features[0]['valence']
    uri=0
  except:
    next

Dropping NAs

In [87]:
new_df = df[-df['track'].isna()]

Merging with Earlier Dataset

In [93]:
new_df2 = pd.merge(total_df2, new_df, how = 'left', left_on = 'TrackURI', right_on = 'TrackURI')

Dropping NAs after merge

In [99]:
new_df2 = new_df2[-new_df2['track'].isna()]

Saving to CSV

In [102]:
new_df2.to_csv('subsampled.csv')

Unfortunately, there were only a few qawwali artists so we'll have to manually download the data for the genres which have fewer instances by looking up albums

In [ ]:
def get_data(playlist, genre):
  playlist_link = playlist
  playlist_URI = playlist_link.split("/")[-1].split("?")[0]
  track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]
  df = pd.DataFrame(track_uris)
  df.columns = ['TrackURI']
  for i in range(df.shape[0]):
    try:
      URI = df.TrackURI[i]
      track = sp.track(track_uris[i])
      features = sp.audio_features(URI)
      df.loc[i, 'Singers'] = track['artists'][0]['name']
      df.loc[i, 'Genre'] = genre
      df.loc[i, 'Artist'] = track['artists'][0]['uri']
      df.loc[i, 'Preview'] = track['preview_url']
      df.loc[i, 'track'] = track['name']
      df.loc[i,'acousticness'] = features[0]['acousticness']
      df.loc[i,'instrumentalness'] = features[0]['instrumentalness']
      df.loc[i,'energy'] = features[0]['energy']
      df.loc[i,'speechiness'] = features[0]['speechiness']
      df.loc[i,'liveness'] = features[0]['liveness']
      df.loc[i,'loudness'] = features[0]['loudness']
      df.loc[i,'danceability'] = features[0]['danceability']
      df.loc[i,'tempo'] = features[0]['tempo']
      df.loc[i,'valence'] = features[0]['valence']
    except:
      next
  
  return df

In [ ]:
qawwali = "https://open.spotify.com/playlist/0USwGwasJVrHVN5Xkdhw0d?si=de90c13a7b4a4d4f"
qawwali2 = "https://open.spotify.com/playlist/0CgbguP9exGwRVdRXLMsPS?si=c902a490d29e4277"
qawwali3 = "https://open.spotify.com/playlist/0CgbguP9exGwRVdRXLMsPS?si=f25dd5cf963048e1"
ghazal = "https://open.spotify.com/playlist/23aE9YFTUW11CcyoQhbGzT?si=79d1f449158d4a01"
ghazal2 = "https://open.spotify.com/playlist/3C50t7049A6xf4w0rhfwgD?si=83083f7f5fc7460b"
edm = "https://open.spotify.com/playlist/2NIe54HdwR5msTjrlHG1Lt?si=ff336b7898a24f98"
metal = "https://open.spotify.com/playlist/27gN69ebwiJRtXEboL12Ih?si=838e862f19144616"


qawwali_df = get_data(qawwali, 'Qawwali') 
qawwal_df2 = get_data(qawwali2, 'Qawwali')
qawwali_df3 = get_data(qawwali3, 'Qawwali') 
ghazal_df = get_data(ghazal, 'Ghazal')
ghazal_df2 = get_data(ghazal2, 'Ghazal')
edm_df = get_data(edm, 'EDM')
metal_df = get_data(metal, 'Metal')

In [ ]:
qawwali_df = pd.concat([qawwali_df, qawwali_df2, qawwali_df3], ignore_index=True)
ghazal_df = pd.concat([ghazal_df, ghazal_df2], ignore_index=True)

In [ ]:
qawwali_df.dropna(inplace = True)
ghazal_df.dropna(inplace = True)
edm_df.dropna(inplace = True)
metal_df.dropna(inplace = True)

In [ ]:
album_data = pd.concat([qawwali_df, ghazal_df, edm_df, metal_df], ignore_index = True)
album_data.to_csv("albumdata.csv")

Concatenating the two

In [8]:
subsampled = pd.concat([subsampled, album_data], ignore_index = True)
subsampled.drop_duplicates(inplace = True)

In [12]:
subsampled['Genre'].value_counts()
subsampled.to_csv('subsampled.csv')

This downloads previews of all songs in dataframe

In [6]:
for i in range(len(a)):
    url = subsampled.loc[i, "Preview"]
    genre = subsampled.loc[i, "Genre"]
    track_name = subsampled.loc[i, "track"]
    track_name = ' '.join(regex.findall('[A-Za-z0-9]+', track_name))
      
    mp3file = requests.get(url)
    os.makedirs(f'./music/{genre}', exist_ok=True)
    with open(f'./music/{genre}/{str(i).zfill(3)}_{track_name.strip(punctuation)}.mp3','wb') as output:
          output.write(mp3file.content)